In [46]:
import os
import json
import geojson
from geojson import FeatureCollection, Feature, Point
import pandas as pd
from pprint import pprint
import numpy as np

In [2]:
# path constructor
def path_constructor(parent, child):
    return os.path.join(parent, child)

In [74]:
# data folder
dataFolder = path_constructor("static", "data")
jsonFolder=path_constructor(dataFolder,"json")
csvFolder=path_constructor(dataFolder,"csv")
jsonFiles = os.listdir(jsonFolder)
geojsonFolder= path_constructor(dataFolder,"geojson")

In [4]:
bird_df = pd.read_csv(path_constructor(csvFolder, "bird_calls.csv"))
bird_df.head()

,fname,label
0,American_Goldfinch_XC114342.wav,American_Goldfinch
1,American_Goldfinch_XC124312.wav,American_Goldfinch
2,American_Goldfinch_XC133564.wav,American_Goldfinch
3,American_Goldfinch_XC133565.wav,American_Goldfinch
4,American_Goldfinch_XC141469.wav,American_Goldfinch


In [5]:
birdID = bird_df.fname.str[-10:-4]
birdList=list(np.unique(bird_df.label))

In [6]:
birdList

['American_Goldfinch',
 'American_Robin',
 'Barn_Swallow',
 'Blue-grey_Gnatcatcher',
 'Blue_Jay',
 'Carolina_Chickadee',
 'Carolina_Wren',
 'Cedar_Waxwing',
 'Northern_Cardinal',
 'Ruby-crowned_Kinglet']

In [7]:
def openFile(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data=json.load(f)
    return data

In [8]:
def birdData(data, birdID):
    datalist = []
    for b in birdID:
        for i in range(len(data['recordings'])):
            if b == data["recordings"][i]["id"]:
                temp=data['recordings'][i]
                datalist.append(temp)
    return datalist

In [9]:
pathlist=[]
for file in jsonFiles:
    path=path_constructor(jsonFolder, file)
    pathlist.append(path)


In [10]:
birdDict={}
for path, bird in zip(pathlist,birdList):
    data=openFile(path)
    blist=birdData(data, birdID)
    birdDict[bird]=blist

In [77]:
def geoBird(chickList):
    bird_keys = ['id', 'en', 'gen', 'sp', 'cnt','loc', 'file', 'date']
    properties=[]
    coordinates = []
    geo = []
    for i in range(len(chickList)):
        prop={k:v for (k,v) in chickList[i].items() if k in bird_keys}
        properties.append(prop)
        lat = chickList[i]["lat"]        
        lng = chickList[i]["lng"]
        if lat != None and lng != None:
            coordinates.append((float(lng),float(lat)))
        
    for pt, prop in zip(coordinates, properties):
        mypoint=Point(pt)
        geo.append(Feature(properties=prop, geometry=mypoint))
    
    feature_collection=FeatureCollection(geo)

    return feature_collection

In [79]:
for k in birdDict.keys():
    geofile=geoBird(birdDict[k])
    filepath=path_constructor(geojsonFolder,f'{k}.json')
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(geofile,f)
    f.close()
    